In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
import sklearn

In [3]:
df=pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler=StandardScaler()

In [8]:
X=scaler.fit_transform(X)

In [9]:
X.shape

(768, 8)

from sklearn.model_selectiom import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [11]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [12]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 [==============================] - 2s 17ms/step - loss: 0.6882 - accuracy: 0.5814 - val_loss: 0.6760 - val_accuracy: 0.5909
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6338 - accuracy: 0.6303 - val_loss: 0.6255 - val_accuracy: 0.6558
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5953 - accuracy: 0.6759 - val_loss: 0.5898 - val_accuracy: 0.6753
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5675 - accuracy: 0.6971 - val_loss: 0.5633 - val_accuracy: 0.7013
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5448 - accuracy: 0.7264 - val_loss: 0.5423 - val_accuracy: 0.7208
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5271 - accuracy: 0.7345 - val_loss: 0.5252 - val_accuracy: 0.7662
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5129 - accuracy: 0.7410 - val_loss: 0.5104 - val_accuracy: 0.7727
Epoch 8/10
20/20 [=

In [14]:
import keras_tuner as kt

**Selecting best Optimizer**

In [15]:
def build_model(hp): #hp :hyperparameter
    
    model= Sequential()
    
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer',['adam','sgd','rmsprop','Adagrad']),loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [16]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3)

In [17]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 06s


In [18]:
 tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [19]:
model=tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(X_train,y_train,batch_size=32,epochs=20,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/20
20/20 [==============================] - 1s 13ms/step - loss: 0.5153 - accuracy: 0.7524 - val_loss: 0.4836 - val_accuracy: 0.8052
Epoch 8/20
20/20 [==============================] - 0s 4ms/step - loss: 0.5040 - accuracy: 0.7590 - val_loss: 0.4761 - val_accuracy: 0.7987
Epoch 9/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4952 - accuracy: 0.7687 - val_loss: 0.4700 - val_accuracy: 0.8052
Epoch 10/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4875 - accuracy: 0.7752 - val_loss: 0.4636 - val_accuracy: 0.8052
Epoch 11/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.7752 - val_loss: 0.4606 - val_accuracy: 0.7987
Epoch 12/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4756 - accuracy: 0.7736 - val_loss: 0.4565 - val_accuracy: 0.8052
Epoch 13/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4711 - accuracy: 0.7736 - val_loss: 0.4546 - val_accuracy: 0.7987
Epoch 14/20
20/

**No of Neurons**

In [22]:
def build_model(hp):
    model=Sequential()
    
    #no of neurons
    units=hp.Int('units',8,128,step=8)
    
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [23]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='/kaggle/working/',
                     project_name='Tuner_files')


In [24]:
tuner.search(X_train,y_train,epochs=3,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 08s


In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [26]:
model=tuner.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dense_1 (Dense)             (None, 1)                 105       
                                                                 
Total params: 1,041
Trainable params: 1,041
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(X_train,y_train,batch_size=32,epochs=20,initial_epoch=4,validation_data=(X_test,y_test))

Epoch 5/20
20/20 [==============================] - 1s 13ms/step - loss: 0.4949 - accuracy: 0.7671 - val_loss: 0.4843 - val_accuracy: 0.8182
Epoch 6/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4799 - accuracy: 0.7687 - val_loss: 0.4725 - val_accuracy: 0.7987
Epoch 7/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4691 - accuracy: 0.7785 - val_loss: 0.4675 - val_accuracy: 0.8052
Epoch 8/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4627 - accuracy: 0.7752 - val_loss: 0.4650 - val_accuracy: 0.8052
Epoch 9/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4585 - accuracy: 0.7785 - val_loss: 0.4624 - val_accuracy: 0.8052
Epoch 10/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4550 - accuracy: 0.7769 - val_loss: 0.4614 - val_accuracy: 0.7922
Epoch 11/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4513 - accuracy: 0.7785 - val_loss: 0.4585 - val_accuracy: 0.7857
Epoch 12/20
20/20

**Selecting Best no of Layers**

In [29]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(units=48,activation='relu',input_dim=8))
    
    #no of layers
    for  i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        
    
        model.add(Dense(units=72,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [30]:
Layertuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='/kaggle/working/',
                     project_name='Tuner_files'
                     )

In [31]:
Layertuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [32]:
Layertuner.get_best_hyperparameters()[0].values

{'units': 104}